In [1]:
import json
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import numpy as np
from sklearn.cluster import DBSCAN
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import haversine_distances
from math import radians
import pickle

In [2]:
# saveout the restaurant and reviews for all restaurants in Reno
with open('df_Reno_restaurants.pkl','rb') as f:
    df_Reno_restaurants = pickle.load(f)
with open('df_Reno_reviews.pkl','rb') as f:
    df_Reno_reviews = pickle.load(f)

In [3]:
import pandas as pd

# Assuming df_restrns_geogroups is your DataFrame
# df_restrns_geogroups = pd.DataFrame(...)

# Sort the DataFrame in descending order of 'review number'
sorted_df = df_Reno_restaurants.sort_values(by='review_count', ascending=False)

# Display the sorted DataFrame
sorted_df[['business_id','review_count']][:50]

target_bid = 'J0joPXxmN-_9Lzafspqdbw'
reviews_oneb = df_Reno_reviews[df_Reno_reviews['business_id'] == target_bid]
df = reviews_oneb.sort_values(by='useful',ascending=False)
df.head()
df['review length'] = df['text'].apply(lambda x: len(x))
review_lengths = list(df['review length'])
review_useful = list(df['useful'])
l=df[df['review_id']=='CRIVPQp5-oxsNFWaCIHSHQ']['text']

target_bid = 'J0joPXxmN-_9Lzafspqdbw'
def filter_reviews(reviews_df, target_bid, N=100):    
    reviews = reviews_df[df_Reno_reviews['business_id'] == target_bid].sort_values(by='useful',ascending=False)
    output = list(reviews[:N]['text'])
    return output
k=filter_reviews(df_Reno_reviews, target_bid, N=20)
len(k)
def flatten_reveiws(reviews_list):
    reviews_paragraph = """
    All the reviews:
    
    """
    for r in reviews_list:
        reviews_paragraph += r
        reviews_paragraph += '\n'
    return reviews_paragraph
o = flatten_reveiws(k[10:])


In [4]:
print(o)


    All the reviews:
    
    This place was pretty busy when I went for a Saturday brunch. Free parking was available at the adjacent parking structure. I like sitting at the counter because you get to witness the bustling and super efficient kitchen. I also like watching my meal being made. For those with purses and bags, there are hooks at the counter so you can hang them! 

As for the food! The orange juice is freshly squeezed so it was a must for me. It can be quite refreshing, especially after a night of drinking in Reno. I opted for the huevos rancheros and my boyfriend the Hawaiian Breakfast plate. 

We both thought that the Hawaiian Breakfast plate was okay. It came with Portuguese sausages (which I usually love but I found these to be a bit bland and salty), fried spam, eggs, and white rice. There was a lot of white rice. 

However, I enjoyed everything about my huevos rancheros. It came in a huge skillet with fried tortillas, eggs, hash browns, pinto beans, cole slaw, and a

In [1]:
from tastefusion.utils.gpt_agent import OpenAIAPI
import yaml
# path = "[deleted text]"
# with open(path, 'r') as f:
#     configs = yaml.safe_load(f)
# check OpenAIAPI class for details of how to configure the openai api using your own credentials.

a = OpenAIAPI(configs)
_ = a.get_prompt_parsing_reviews(o)


ModuleNotFoundError: No module named 'tastefusion.utils.gpt_agent'

In [9]:
# current prompt
print(a.prompt_head_parse_reviews)


        I will give you a list of reviews of a restaurant, and please answer the following questions base don the reviews I give you:

            1. tell me 2 things about this restaurant that people like about it the most.
            2. good for breakfast/brunch? answer [yes] or [not mentioend]
            3. good for lunch? answer [yes] or [not mentioend]
            4. good for dinner? answer [yes] or [not mentioend]
            5. Based on people's reviews, is it good for group of friends? answer [yes] or [not mentioend]
            6. use 10 words to describe what categories of foods are offered at the restaurant that is not characterized by the previouse questions.
            7. use 10 words to describe the taste categories of the foods offered att he restaurant.
            8. give top 10 recommended foods.
            9. top 2 not recommended foods.
            10. use 3 words to describe the accomoations for food resstrictions. If not clearly indicated, say [not mentioned]

In [5]:
# new
from openai import OpenAI

client = OpenAI(
  api_key=configs['api_key'],  # this is also the default, it can be omitted
)

### Adjusting prompts, and try differnt models.
Decided on the gpt-3.5-turbo-1106 model

In [42]:

stream = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[{"role": "user", "content": a.curr_prompt_parsing_reviews}],
    stream=False,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

AttributeError: 'tuple' object has no attribute 'choices'

In [61]:
text = stream.choices[0].message.content


In [62]:
print(text)

1. People like the bustling and super efficient kitchen and the freshly squeezed orange juice the most.
2. [not mentioned]
3. [yes]
4. [not mentioned]
5. [yes]
6. Skillet breakfasts, oatmeal, fresh OJ, corned beef hash, benedicts, pancakes, biscuits and gravy, French toast, Monte Cristos, hash browns
7. Homey, cozy, classic, hearty, comfort, spicy, tangy, rich, flavorful, satisfying
8. Sierra Omelette, Hawaiian Breakfast Plate, Pancake Sandwich, Chef George Benedict, crab cake Benedict, huevos rancheros, corned beef hash and eggs, Monte Cristo, waffles with bananas foster, French toast
9. Fried spam, Philly omelet special
10. [Not mentioned]


In [27]:

stream = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[{"role": "user", "content": a.curr_prompt_parsing_reviews}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

1. Emphasis on quality and value, big portions
2. [yes]
3. [yes]
4. [yes]
5. [yes]
6. Omelettes, benedicts, scrambles, skillet breakfasts, pancakes, biscuits and gravy, French toast, Monte Cristos, Loco Moco
7. Spicy, tangy, sweet, flavorful, fresh, crispy, crunchy, rich, lemony, smooth
8. Sierra Omelette, Huevos Rancheros, Hawaiian Breakfast plate, French Toast, Hash Browns, Crab Cake Benedict, Pancake Sandwich, Eggs Benedict, Ham and Cheese Omelette, Chef George Benedict
9. Hawaiian Breakfast plate, Philly Omelet
10. [not mentioned]

In [28]:

stream = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": a.curr_prompt_parsing_reviews}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

1. [Efficient service], [Generous portions]
2. [yes]
3. [not mentioned]
4. [not mentioned]
5. [yes]
6. [Hawaiian], [Mexican], [Omelettes], [Pancakes], [French Toast], [Skillet breakfasts], [Egg dishes], [Hash browns], [Biscuits and gravy], [Monte Cristos]
7. [Fresh], [Savory], [Spicy], [Sweet], [Flavorful], [Addicting], [Bland], [Hearty], [Comforting], [Decadent]
8. [Huevos Rancheros], [Sierra Omelette], [California Omelette], [Loco Moco], [Tamale and Eggs], [Corned Beef Hash and Eggs], [Pancake Sandwich], [Eggs Benedict], [Chef George Benedict], [Crab Cake Benedict]
9. [Portuguese sausages], [Philly omelet special]
10. [not mentioned]

In [8]:
stream = client.chat.completions.create(
    model="gpt-4-0613",
    messages=[{"role": "user", "content": a.curr_prompt_parsing_reviews}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

1. [Efficient kitchen], [Friendly service]
2. [yes]
3. [not mentioned]
4. [not mentioned]
5. [not mentioned]
6. [Huevos Rancheros], [Hawaiian Breakfast], [Sierra Omelette], [California Omelette], [Pancake Sandwich], [Eggs Benedict], [French Toast], [Hash Browns], [Country Fried Steak], [Corned Beef Hash]
7. [Freshly squeezed orange juice], [Flavorful sauce], [Spicy], [Large portions], [Sweet], [Salty], [Rich], [Crispy], [Savory], [Decadent]
8. [Huevos Rancheros], [Hawaiian Breakfast], [Sierra Omelette], [California Omelette], [Pancake Sandwich], [Eggs Benedict], [French Toast], [Hash Browns], [Country Fried Steak], [Corned Beef Hash]
9. [Portuguese sausages], [Spicy chicken filling in Sierra Omelette]
10. [not mentioned]

In [7]:
stream = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[{"role": "user", "content": a.curr_prompt_parsing_reviews}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

1. The things people like most about this restaurant are watching the bustling and efficient kitchen and the freshly squeezed orange juice.
2. [yes]
3. [not mentioned]
4. [not mentioned]
5. [yes]
6. Egg dishes, benedicts, omelettes, scrambles, skillet breakfasts, pancakes, biscuits and gravy, French Toast, Monte Cristos, Hawaiian breakfast options
7. Spicy, complex, tangy, rich, savory, flavorful, traditional, lusciously rich, lemony, smooth
8. Huevos Rancheros, Hawaiian Breakfast Plate, Sierra Omelette, Pancake Sandwich, Eggs Benedict, Loco Moco, Tamale and Eggs, Corned Beef Hash and Eggs, French Toast, Crab Cake Benedict
9. California Omelette, Philly Omelet Special
10. [not mentioned]

In [6]:
stream = client.chat.completions.create(
    model="gpt-4-0613",
    messages=[{"role": "user", "content": a.curr_prompt_parsing_reviews}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

1. [Efficient service], [Generous food portion]
2. [yes]
3. [not mentioned]
4. [not mentioned]
5. [not mentioned]
6. [American diner], [Mexican], [Hawaiian], [Fresh fruits], [French toasts], [Egg omelettes], [Bacon], [Hash browns], [Corned Beef Hash], [Crab cakes]
7. [Flavorful], [Spicy], [Refreshing], [Sweet], [Crunchy], [Smooth], [Crispy], [Custardy], [Savory], [Tangy]
8. [Huevos rancheros], [Hawaiian Breakfast plate], [Sierra Omelette], [California Omelette], [Tamale and Eggs], [Corned Beef Hash and Eggs], [Pancake Sandwich], [Country Fried-steak with eggs], [Waffles with bananas foster topping], [Chef George Benedict]
9. [Portuguese sausages], [Philly omelet special]
10. [not mentioned]

In [6]:
stream = client.chat.completions.create(
    model="gpt-4-0613",
    messages=[{"role": "user", "content": a.curr_prompt_parsing_reviews}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

1. [Efficiency and friendliness of the service], [large portion sizes], [tasty food], [atmosphere of the diner]
2. [yes]
3. [not mentioned]
4. [not mentioned]
5. [not mentioned]
6. [Breakfast], [Brunch], [American], [Mexican], [Tex-Mex], [Classic Diner], [Omelettes], [Eggs Benedict], [Hash Browns], [French Toast]
7. [Flavorful], [Savory], [Sweet], [Crispy], [Custardy], [Refreshing], [Satisfying], [Comfort Food], [Rich], [Lemon]
8. [Huevos Rancheros], [Pancake Sandwich], [Sierra Omelette], [Waffles with Bananas Foster Topping], [Chef George Benedict], [French Toast], [Hash Browns], [Crab Cake Benedict], [Loco Moco], [Corned Beef Hash and Eggs]
9. [Hawaiian Breakfast plate], [Philly Omelette special]
10. [not mentioned]

In [20]:

stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": a.curr_prompt_parsing_reviews}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

1. People like the atmosphere and efficiency of the kitchen at the restaurant. 
2. Not mentioned
3. Not mentioned
4. Not mentioned
5. Yes
6. Omelettes, scrambles, skillet breakfasts, pancakes, biscuits and gravy, French Toast, Monte Cristos, Hawaiian breakfast options
7. Spicy, flavorful, tangy, chewy, cool, warm, crunchy, gooey, meaty
8. Sierra Omelette, Hawaiian Breakfast plate, Huevos Rancheros, Pancake Sandwich, Eggs Benedict, Corned Beef Hash and Eggs, Tamale and Eggs, California Omelette, Waffles with bananas foster topping, Chef George Benedict
9. California Omelette, Hawaiian Breakfast plate
10. Not mentioned

In [21]:
stream = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[{"role": "user", "content": a.curr_prompt_parsing_reviews}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

1. People like: the freshly squeezed orange juice and the friendly, fast service
2. [yes]
3. [not mentioned]
4. [not mentioned]
5. [not mentioned]
6. Huevos rancheros, Hawaiian Breakfast, Sierra Omelette, California Omelette, Pancake Sandwich, Tamale and Eggs, Corned Beef Hash and Eggs, Loco Moco, Chef George Benedict, French toast
7. Spicy, Tasty, Savory, Tangy, Flavorful, Decadent, Refreshing, Delightful, Luscious, Delicious
8. Huevos rancheros, Sierra Omelette, French toast, Pancake Sandwich, Tamale and Eggs, Corned Beef Hash and Eggs, Loco Moco, Chef George Benedict, Crab Cake Benedict, Orange juice
9. Portuguese sausages, Eggs Benedict
10. [not mentioned]

In [6]:
stream = client.chat.completions.create(
    model="gpt-4-0613",
    messages=[{"role": "user", "content": a.curr_prompt_parsing_reviews}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

1. Efficiency and friendliness of the service, large portion sizes, quality of the coffee, interesting menu options
2. [yes]
3. [not mentioned]
4. [not mentioned]
5. [not mentioned]
6. Mexican, Hawaiian, American, Breakfast, Brunch, Eggs, Pancakes, French Toast, Benedict, Hash Browns
7. Flavorful, Hearty, Fresh, Balanced, Cinnamon, Custardy, Crispy, Decadent, Lemony, Savory
8. Huevos Rancheros, Hawaiian Breakfast Plate, Sierra Omelette, Pancake Sandwich, Waffle with Bananas Foster Topping, Chef George Benedict, French Toast, Hash Browns, Crab Cake Benedict, Tamale and Eggs
9. Philly Omelette Special, Country Fried-Steak with eggs
10. [not mentioned]

In [10]:
stream = client.chat.completions.create(
    model="gpt-4-0613",
    messages=[{"role": "user", "content": a.curr_prompt_parsing_reviews}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

1. Mexican, Hawaiian, American, Diner, Breakfast
2. Flavorful, Savory, Fresh, Spicy, Sweet
3. Huevos Rancheros, Sierra Omelette, Pancake Sandwich, Waffle with Bananas Foster Topping, Chef George Benedict
4. Hawaiian Breakfast plate, Philly Omelette special 
5. Efficiency and friendliness of the service, the portion sizes
6. [Yes]
7. [Yes]
8. [Not mentioned]
9. [Not mentioned]
10. [Not mentioned]

In [6]:
stream = client.chat.completions.create(
    model="gpt-4-0613",
    messages=[{"role": "user", "content": a.curr_prompt_parsing_reviews}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

1. Efficiency and friendliness of the service, the generous portion sizes.
2. [Yes]
3. [Not mentioned]
4. [Not mentioned]
5. [Yes]
6. Mexican, Hawaiian, American, Diner, Breakfast
7. Flavorful, Savory, Fresh, Spicy, Sweet
8. Huevos Rancheros, Sierra Omelette, Pancake Sandwich, Waffle with Bananas Foster Topping, Chef George Benedict
9. Hawaiian Breakfast plate, Philly Omelette special 
10. [Not mentioned]

In [8]:
stream = client.chat.completions.create(
    model="gpt-4-0613",
    messages=[{"role": "user", "content": a.curr_prompt_parsing_reviews}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

1. Efficiency and friendliness of the service, the portion sizes, fresh and flavorful food, and the busy yet comforting atmosphere.
2. [yes]
3. [not mentioned]
4. [not mentioned]
5. [yes]
6. American, Mexican, Hawaiian, Eggs, Pancakes, Spicy chicken, Waffles, Toast, Crab cakes, Omelettes
7. Savory, Flavorful, Sweet, Spicy, Fresh, Chewy, Smooth, Tangy, Custardy, Crunchy
8. Huevos Rancheros, Sierra Omelette, Pancake Sandwich, Waffle with Bananas Foster topping, French Toast, Hash Browns, Crab Cake Benedict, Eggs Benedict, Country Fried-Steak with Eggs, Tamale and Eggs
9. Hawaiian Breakfast Plate, Philly Omelette Special 
10. [not mentioned]

In [22]:
stream = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[{"role": "user", "content": a.curr_prompt_parsing_reviews}],
    stream=False,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

AttributeError: 'tuple' object has no attribute 'choices'

In [24]:
type(stream)

openai.types.chat.chat_completion.ChatCompletion